In [1]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
import pandas as pd
octave.addpath('C:/dynare/6.1/matlab')  
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

    _pyeval at line 57 column 30

    _pyeval at line 57 column 30



'.;C:\\Users\\blankenw\\AppData\\Local\\anaconda3\\Lib\\site-packages\\oct2py;C:\\Users\\blankenw\\AppData\\Local\\anaconda3\\Lib\\site-packages\\octave_kernel;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\9.1.0\\site\\oct\\x86_64-w64-mingw32;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\site\\oct\\api-v59\\x86_64-w64-mingw32;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\site\\oct\\x86_64-w64-mingw32;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\9.1.0\\site\\m;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\api-v59\\m;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\m;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\m\\once_only;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\share\\octave\\site\\m\\startup;C:\\Program Files\\GNU Octave\\Octave-9.1.0\\mingw64\\lib\\octave\\9.1.0\\oct\\x86_64-w64-mingw32;C:\\Program Fi

In [2]:
n_goods=3

In [3]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=10000, irf=100);")

In [4]:
%run make_model.ipynb
%run make_vars.ipynb
%run make_parameters_and_shocks.ipynb
%run make_steady_state.ipynb


Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [5]:
octave.dynare('model_1.mod')
oo = octave.pull('oo_')
mm = octave.pull('M_')

Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 12 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
7 block(s) found:
  6 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m00s.

resid: The initial values for the steady state of the following variables are complex:
n0
s0


Residuals of the static equations:
Equat

Oct2PyError: Octave evaluation error:
error: The steadystate file did not compute the steady state
error: called from:
    print_info at line 33, column 5
    steady at line 143, column 5
    driver at line 567, column 1
    dynare at line 310, column 5

In [ ]:
names1 = [attr for attr in dir(oo) if not attr.startswith('__')]

Now we want to create a class where each each attribute in oo becomes an attribute of this class. So far we take only one item from mm. This is a list of endogenous variables. Below we define the class. 

In [ ]:
class from_octave:
	def __init__(self, oo_structure, mm_structure, attribute_names):
		self.oo_structure = oo_structure
		self.mm_structure = mm_structure
		self. attribute_names =  attribute_names
		self.create_attributes()
		self.make_list()
     
	def create_attributes(self): 
		for attr in self.attribute_names:
			setattr(self, attr, getattr(self.oo_structure, attr)) #this is the attributes we pull from the oo structure. We are making them attributes of the class
	
	def make_list(self): 
		self.names = list(getattr(self.mm_structure, 'endo_names')) #getting one item from the mm structure. This is the endo_names
		self.names = [item[0] for item in self.names] #pulling mm gives us a list of cell arrays. We are pulling the first item from each cell array

We see that we now can reference any of the data from `oo` with the `tt.` notation. Some examples are below. 

In [ ]:
tt = from_octave(oo, mm, names1)

In [ ]:
dir(tt)

We will care about the irfs but these are also of type `oct2py.io.Struct`. We convert it as above. We will likley make this part of the above class later, but for now we create a new class. 

In [ ]:
names2 = [attr for attr in dir(tt.irfs) if attr.endswith('shock')]
#names2 = [x for _, x in sorted(zip(tt.names, names2), key=lambda pair: pair[0][0])]

I will put more desciptors here later. However, it is getting the impulse responses as difference and levels. It is simultanosuly creating two dataframes containing all differences and levels data.  

In [ ]:
#recall make_levels is created in simpler ffolder
import numpy as np
import pandas as np
class make_irfs:
	def __init__(self, dta, vars, names):
		self.data = dta
		self.vars = vars
		self.names = names
		self.create_difs()
		
  	
	def create_difs(self):
		for var in self.vars:
			setattr(self, var, pd.DataFrame())  # Initialize an empty DataFrame with the dynamic name
			list_short = [name for name in self.names if name.startswith(var)]
			for attr in list_short:
				x = getattr(self.data, attr)
				x = x.flatten()
				getattr(self, var)[attr] = x  # Assign the flattened array to the DataFrame
				new_columns = [col.replace(var + '_', '').replace('shock', '') for col in getattr(self, var).columns]
				getattr(self, var).columns = new_columns
    
    



In [ ]:
varlist=['c','i','y','k','n0','s0']

In [ ]:
ir.data

In [ ]:
ir = make_irfs(tt.irfs,varlist, names2)
print(ir.__dict__.keys())

In [ ]:
dir(ir)

Lets get a sense of what we have. 

This will also need more documentation, but it is getting the results of dynare's simulation. It defines the individual series and also puts all in a dataframe. 

In [ ]:
import numpy as np

class make_data:
	def __init__(self, dta,names):
		self.data = dta
		self.names = names
		self.make_levels()
		self.make_raw()

	def make_raw(self):
		self.df_raw = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			new_attr =attr + 'r'
			raw_level = self.data[j]
			setattr(self, new_attr, raw_level)
			self.df_raw[new_attr] = raw_level
			
  		
	def make_levels(self):
		self.df = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			raw_level = self.data[j]
			if attr != 'i' and 'theta' not in attr:
				current_level=np.exp(raw_level)
			else:
				current_level=raw_level
			setattr(self, attr, current_level)
			self.df[attr] = current_level  # Store in DataFrame



In [ ]:
eg=make_data(tt.endo_simul,tt.names)

In [ ]:
eg.df

In [ ]:
eg.df.describe()

In [ ]:
def x_find(g,tt):
    thetag='theta'+str(g)
    ag='a'+str(g)
    omegag='omega'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    
    
    ai=eg.df[ag][tt]
    a0=eg.df['a0'][tt]
    k=eg.df['k'][tt-1]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    theta0=eg.df['theta0'][tt]
    eps=params['eps']
    omegai=params[omegag]
    omega0=params['omega0']
    
    x=(k*s0/n0)**((theta0-thetai)*(1-eps))*(omegai/omega0)**eps* \
    ((a0)**(1-theta0)/(ai)**(1-thetai)*(theta0/thetai)**thetai*((1-theta0)/(1-thetai))**(1-thetai))**(1-eps)
    
    return x

In [ ]:
def X_find(t):
    X=0
    for g in range(0,n_goods):
        X+=x_find(g,t)
    return X

In [ ]:
def Z_find(t):
    Z=0
    for g in range(0,n_goods):
        thetag='theta'+str(g)
        thetai=eg.df[thetag][t]
        Z+=x_find(g,t)*thetai
    return Z

In [ ]:
eg.df['X'] = [X_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Z'] = [Z_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Q']=(eg.df['Z']-eg.df['theta0']*eg.df['X'])/eg.df['X']

In [ ]:
def py_n(tt,g):
    
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    a0=eg.df['a0'][tt]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    k=eg.df['k'][tt-1]
    prod=(1-theta0)*a0**(1-theta0)*k**theta0*(s0/n0)**theta0/(1-thetai)
    return prod

In [ ]:
def n_i(tt,g):
    
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    y=eg.df['y'][tt]
    c=eg.df['c'][tt]
    X=eg.df['X'][tt]
    Q=eg.df['Q'][tt]
    
    xi=x_find(g,tt)
    if g>0:
        ni=xi/X*c/y*(1-thetai)/(1-theta0-c/y*Q)
    else:
        ni=(1-theta0)*(1-c/y*(1-xi/X))/(1-theta0-c/y*Q)
    return ni

In [ ]:

def within_find(tt):
    within=0
    ylag=eg.df['y'][tt-1]
    a=eg.df['a0'][tt]/eg.df['a0'][tt-1]
    for g in range(0,n_goods):
        withing=(a*py_n(tt,g)-py_n(tt-1,g))*n_i(tt-1,g)
        within+=withing
    return within/ylag

In [ ]:

def static_find(tt):
    static=0
    N=0
    N_minus=0
   
    ylag=eg.df['y'][tt-1]
    
    for g in range(0,n_goods):
       
        staticg=(n_i(tt,g)-n_i(tt-1,g))*py_n(tt-1,g)
        
        
        N+=n_i(tt,g)
        N_minus+=n_i(tt-1,g)
        #if g==n_goods-1:
            #(print('check1', N,N_minus))
        if g==0:
            (print('check2',n_i(tt,g)-eg.df['n0'][tt]))
        static+=staticg
        
    return static/ylag

In [ ]:
def dynamic_find(tt):
    dynamic=0
    ylag=eg.df['y'][tt-1]

    a=eg.df['a0'][tt]/eg.df['a0'][tt-1]
    for g in range(0,n_goods):
        
        dynamicg=(a*py_n(tt,g)-py_n(tt-1,g))*(n_i(tt,g)-n_i(tt-1,g))
        dynamic+=dynamicg
    return dynamic/ylag

In [ ]:

eg.df['grow'] = (eg.df['y']*eg.df['a0']/eg.df['a0'].shift(1)-eg.df['y'].shift(1))/eg.df['y'].shift(1)
eg.df.loc[0, 'grow'] = np.nan
eg.df['within'] = [within_find(t) if t > 1 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]
eg.df['static'] = [static_find(t) if t > 1 and t < (eg.df.shape[0]-1) else np.nan for t in range(eg.df.shape[0])]
eg.df['dynamic'] = [dynamic_find(t) if t > 1 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]
eg.df['check'] = eg.df['grow']-eg.df['within']-eg.df['static']-eg.df['dynamic']


In [ ]:
eg.df[100:200]

In [ ]:

eg.df.shape[0]

In [ ]:
eg.df.describe()

In [ ]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov